In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Read in the address dataset
address= pd.read_csv('address_dataset.csv')
address.head()

,employee_ID,first_name,last_name,relationship,address,city,state,zip,phone1
0,4069432,Fe,Denoia,Employee,38 W Vickery Blvd,Bound Brook,NJ,8805,732-655-9726
1,4827171,Timika,Biscardi,Spouse,682 Pinehurst St,Los Angeles,CA,90067,310-289-3570
2,9173685,Maryalice,Natsis,Employee,39 E 19th Ave,Caro,MI,48723,989-403-4187
3,7859119,Filomena,Stirewalt,Employee,287 S Date Ave #1,Medford,MA,2155,781-231-5265
4,1988592,Anjanette,Wortley,Employee,53 30w W,Aiea,HI,96701,808-531-3801


In [4]:
# Read in the email dataset
email= pd.read_csv('email_dataset.csv')
email.head()

,employee_ID,email
0,4453134,tmillender@hotmail.com
1,1875342,NaN
2,4163360,NaN
3,2346489,lhutchison@hutchison.com
4,1195910,cplatania@yahoo.com


## Inspecting Address Data

In [5]:
address.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48732 entries, 0 to 48731
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   employee_ID   48732 non-null  int64 
 1   first_name    48732 non-null  object
 2   last_name     48732 non-null  object
 3   relationship  48732 non-null  object
 4   address       48732 non-null  object
 5   city          48732 non-null  object
 6   state         48732 non-null  object
 7   zip           48732 non-null  int64 
 8   phone1        48732 non-null  object
dtypes: int64(2), object(7)
memory usage: 3.3+ MB


### Notes

-There are no null values 

-All datatypes are logical for each attribute except phone

-Phone is an object where as we may want it to be an int64. It just depends on our needs. For now we can leave it as an object

-There are 48732 observations

In [6]:
# How many unique values are there for each attribute
address.nunique()

employee_ID     48732
first_name       5162
last_name       37683
relationship        2
address         47350
city             1494
state              51
zip              2861
phone1          48729
dtype: int64

### Notes

-There are 48732 unique employee_ids which is good

-There are not enough unique phone numbers which tells me that some spouses likely listed he same phone number as their spouse

## Inspecting the Email Dataset

In [7]:
email.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50625 entries, 0 to 50624
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   employee_ID  50625 non-null  int64 
 1   email        44192 non-null  object
dtypes: int64(1), object(1)
memory usage: 791.1+ KB


In [8]:
email.nunique()

employee_ID    50625
email          43056
dtype: int64

### Notes

-We have no null values in emloyee_id

-We have roughly 6,000 null values for email

-We have only 43056 unique email addresses which means that some email addresses are shared between those with different employee_ids

-We have more employee_ids in this dataset than we do in the addresses dataset leading me to beleive may be missing some data in that data file

In [9]:
# How many null values are there in the email column of the email dataset?
print('We have {} null emails in the email datatset.'.format(email['email'].isnull().sum()))

We have 6433 null emails in the email datatset.


In [10]:
# Create a dataframe to store all the employees with no email address in the email dataframe
null_emails= email[email['email'].isnull()]
null_emails

,employee_ID,email
1,1875342,NaN
2,4163360,NaN
5,6253152,NaN
11,4025922,NaN
19,867601,NaN
...,...,...
50592,7260497,NaN
50608,4674893,NaN
50609,5564598,NaN
50615,8974403,NaN


In [11]:
# create a dataframe of those with all double emails
double_email= email[email.duplicated(['email'])]
# Drop the NaNs to just see those with duplicated emails
double_email= double_email.dropna()
# Sort by email address
double_email= double_email.sort_values(by='email')
print('We have {} employees in the email datatset with duplicate emails.'.format(len(double_email)))

We have 1136 employees in the email datatset with duplicate emails.


In [12]:
# We have more employee_ids in the email df than in the address df
# Who is in the email df but not in the address df? 
# Create list of all emp_ids in the email dataset
email_emp_id= list(email['employee_ID'])
# Create list of all the emp_ids in the address dataset
address_emp_id= list(address['employee_ID'])
# Create empty list
no_address= []

# Loop through emp_id in email
for x in email_emp_id:
    # if there is not corresponding emp_id in the address df then add that emp_id to the empty list from above
    if x not in address_emp_id:
        no_address.append(x)

print('There are {} employee IDs in the email dataset that are not in the address dataset.'.format(len(no_address)))

There are 2413 employee IDs in the email dataset that are not in the address dataset.


In [13]:
# Create a df of those emp_ids which are present in the email_dataset but not the adress_dataset
no_address_df= email[email['employee_ID'].isin(no_address)]
no_address_df

,employee_ID,email
1,1875342,NaN
5,6253152,NaN
19,867601,NaN
26,3211378,NaN
32,7090560,NaN
...,...,...
50514,5347141,NaN
50534,9281782,NaN
50538,3993520,NaN
50545,8267851,NaN


In [26]:
# Are there any emplyee's without addresses who do have email's? 
no_address_df.isnull().sum()

employee_ID       0
email          2413
dtype: int64

## Joining the Datasets

In [15]:
# Merge the datasets but performing a left join with address and email joining on the email column
# By doing this we will not have an emails for those we don't have address info on
df= pd.merge(left=address, right= email, how='left', on='employee_ID')

In [16]:
df.head()

,employee_ID,first_name,last_name,relationship,address,city,state,zip,phone1,email
0,4069432,Fe,Denoia,Employee,38 W Vickery Blvd,Bound Brook,NJ,8805,732-655-9726,fe_denoia@gmail.com
1,4827171,Timika,Biscardi,Spouse,682 Pinehurst St,Los Angeles,CA,90067,310-289-3570,tbiscardi@gmail.com
2,9173685,Maryalice,Natsis,Employee,39 E 19th Ave,Caro,MI,48723,989-403-4187,mnatsis@hotmail.com
3,7859119,Filomena,Stirewalt,Employee,287 S Date Ave #1,Medford,MA,2155,781-231-5265,filomena.stirewalt@stirewalt.org
4,1988592,Anjanette,Wortley,Employee,53 30w W,Aiea,HI,96701,808-531-3801,anjanette_wortley@wortley.org


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48732 entries, 0 to 48731
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   employee_ID   48732 non-null  int64 
 1   first_name    48732 non-null  object
 2   last_name     48732 non-null  object
 3   relationship  48732 non-null  object
 4   address       48732 non-null  object
 5   city          48732 non-null  object
 6   state         48732 non-null  object
 7   zip           48732 non-null  int64 
 8   phone1        48732 non-null  object
 9   email         44192 non-null  object
dtypes: int64(2), object(8)
memory usage: 4.1+ MB


In [18]:
df.nunique()

employee_ID     48732
first_name       5162
last_name       37683
relationship        2
address         47350
city             1494
state              51
zip              2861
phone1          48729
email           43056
dtype: int64

In [28]:
# create a dataframe of those with a duplicated email
duplicated_email= df[df.duplicated(['email'])]
# Drop the NaNs to just see those with duplicated emmails
duplicated_email= duplicated_email.dropna()
# Sort by email address
duplicated_email= duplicated_email.sort_values(by='email')
duplicated_email.head()

,employee_ID,first_name,last_name,relationship,address,city,state,zip,phone1,email
21799,1139325,Aaron,Smutny,Employee,93316 S 2nd Ave #47,Los Angeles,CA,90048,323-383-1267,aaron@cox.net
45758,5210699,Aaron,Rolin,Employee,13955 E Campbell Ave,Nutley,NJ,7110,973-260-1301,aaron@cox.net
43119,5619161,Adrian,Hachez,Spouse,65 Worth St,Columbia,SC,29201,803-394-7240,adrian@hotmail.com
25353,2428173,Adrian,Toquinto,Employee,37 W 19th St,College Station,TX,77840,979-576-4870,adrian@hotmail.com
44609,7120247,Adrien,Poskus,Employee,57 Castor Ave #35,Chicago,IL,60605,312-767-8144,adrien@yahoo.com


In [20]:
# How many duplicate email addresses are there in the merged dataset?
print("We have {} duplicate emails in the merged dataset.".format(len(duplicated_email)))

We have 1136 duplicate emails in the merged dataset.


In [21]:
# How many null values are there in the email column of the merged dataset
print('We have {} null emails in the merged datatset.'.format(df['email'].isnull().sum()))

We have 4540 null emails in the merged datatset.


In [22]:
# Create a dataframe to hold those who have no email address in the merged dataframe
merged_no_email= df[df['email'].isnull()]
merged_no_email

,employee_ID,first_name,last_name,relationship,address,city,state,zip,phone1,email
10,6137607,Winfred,Lammy,Employee,518 S Rainbow Blvd Bldg,Flanders,NJ,7836,973-320-5873,NaN
31,9606969,Ena,Villena,Employee,56469 S State St,Hyattsville,MD,20785,301-683-4137,NaN
68,9297568,Keshia,Bernardo,Employee,7 E 29th St #8794,San Leandro,CA,94577,510-879-3634,NaN
82,3718867,Cristine,Hennis,Employee,43107 Seneca Rd #5,Bowling Green,KY,42104,270-359-2272,NaN
127,6477916,Venus,Millisor,Employee,473 E Main St #3443,Indianapolis,IN,46236,317-532-1797,NaN
...,...,...,...,...,...,...,...,...,...,...
48727,514171,Ahmed,Truong,Employee,70421 Wyatt Dr,Dallas,TX,75207,214-630-6209,NaN
48728,5055739,Page,Zolocsik,Employee,9 W Belmont Ave,San Diego,CA,92110,858-640-8928,NaN
48729,5335981,Idalia,Yashinski,Employee,11987 S Fair Oaks Ave,San Francisco,CA,94107,415-797-6394,NaN
48730,3471421,Samira,Atengco,Employee,301 Bannock St,Wappingers Falls,NY,12590,845-990-4883,NaN


# Summary for Contact

### Issues with the datasets

1) There are 6,433 employee_IDs with null email addresses in the email_dataset. Those employees missing an email are stored in the CSV file entitled "Null_Emails"

2) There are 1136 employees in the email datatset with duplicate email addresses. Those employees are held in the CSV file entitled "Double_Email".

3) There are 2413 employee_IDs in the email_dataset that do not have a match in the address_dataset based on employee_ID. Those employees are held in the CSV file entitled "No_Address".

In [23]:
null_emails.to_csv("Null_Emails")
double_email.to_csv("Double_Email")
no_address_df.to_csv("No_Address")

# Internal Summary for Coworkers

## Summary of my work

I have joined the email_datatset and the address_datatset by matching on the employee_id. This merged dataset can be found in the CSV file entitled "Merged_DF". I did a left join with address_dataset and email_address so any employees from the email_dataset not found in the address_dataset were dropped. I did this becuase those employees in the email_dataset but not the address_dataset also happened to have Null values for their email addresses as well. No employees were lost from the address_dataset during this merge.



In [24]:
df.to_csv("Merged_DF")

## Issues with the merged Dataset

1) There are 1136 duplicate emails in the merged dataset. Those employees can be found in the CSV entitled "Duplicate_Emails".

2) There are 4540 null emails in the merged datatset.Those employees with no email address can be found in the CSV entitled "Merged_No_Email".

In [25]:
duplicated_email.to_csv("Duplicate_Emails")
merged_no_email.to_csv("Merged_No_Email")